<a href="https://colab.research.google.com/github/MDankloff/Cluster-Bias-Disco/blob/main/baf_exploratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/feedzai/bank-account-fraud.git
!git clone https://github.com/MDankloff/Cluster-Bias-Disco.git

Cloning into 'bank-account-fraud'...
remote: Enumerating objects: 110, done.
remote: Counting objects: 100% (110/110), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 110 (delta 54), reused 64 (delta 23), pack-reused 0 (from 0)
Receiving objects: 100% (110/110), 1.13 MiB | 8.70 MiB/s, done.
Resolving deltas: 100% (54/54), done.
Cloning into 'Cluster-Bias-Disco'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 54 (delta 28), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (54/54), 446.32 KiB | 5.01 MiB/s, done.
Resolving deltas: 100% (28/28), done.


In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgbm
import yaml              # Read hyperparameter space configuration
import glob
import os
import joblib

from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, roc_auc_score, roc_curve
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, LabelEncoder # Categorical encoding for LGBM models
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn import metrics

#from aequitas.group import Group                # Fairness metrics
#from random_search import RandomValueTrial, suggest_callable_hyperparams  # Random search wrapper methods

import yaml

# If the file is within a subdirectory, for example, named "config":
with open("/content/lightgbm_hyperparameter_space.yaml", "r") as file:
    # Remove the 'Loader' argument
    hyperparam_space = yaml.safe_load(file)

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
base_path = '/content/drive/MyDrive/Mirthe Supervision Map /Paper 3: FC Bias Disco/BAF/parquet data'

extension = "parquet" #for smaller "parquet" depending on the downloaded file
data_paths = glob.glob(f"{base_path}/*.{extension}")

def read_dataset(path, ext = extension):
    if ext == "csv":
      return pd.read_csv(path)
    elif ext == "parquet":
      return pd.read_parquet(path)
    else:
      raise ValueError(f"Unsupported file extension: {ext}")

# Extract variant name from the file path (without the extension)
def get_variant(path):
    return os.path.basename(path).split(".")[0]

# Dictionary comprehension to read all CSV files into a dictionary of DataFrames
dataframes = {
    get_variant(path): read_dataset(path) for path in data_paths
}
print(f"Loaded datasets: {list(dataframes.keys())}")

datasets_paths = {
    "Base": base_path + "/Base.parquet", # sampled to best represent original dataset
    "Variant I": base_path + "/Variant I.parquet", # higher group size disparity than base - reducing the size of the minority group from approx 20 - 10% of the dataset
    "Variant II": base_path + "/Variant II.parquet", # higher prevalence disparity than base - one group has 5 x the fraud detection rate of the other while group sizes are equal
    "Variant III": base_path + "/Variant III.parquet", # better separability for one of the groups -
    "Variant IV": base_path + "/Variant IV.parquet", # higher prevalence disparity in train
    "Variant V": base_path + "/Variant V.parquet", # better separability in train for one of the groups
}

Loaded datasets: ['Base', 'Variant I', 'Variant II', 'Variant III', 'Variant IV', 'Variant V']


In [21]:
base = dataframes['Base']
variant1 = dataframes['Variant I']
variant2 = dataframes['Variant II']
variant3 = dataframes['Variant III']
variant4 = dataframes['Variant IV']
variant5 = dataframes['Variant V']

In [ ]:
'''categorical_features = [
    "payment_type",
    "employment_status",
    "housing_status",
    "source",
    "device_os",
]'''

In [27]:
variant1.info()
variant1['fraud_bool'].unique()
variant1['payment_type'].unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 32 columns):
 #   Column                            Non-Null Count    Dtype  
---  ------                            --------------    -----  
 0   fraud_bool                        1000000 non-null  int64  
 1   income                            1000000 non-null  float64
 2   name_email_similarity             1000000 non-null  float64
 3   prev_address_months_count         1000000 non-null  int64  
 4   current_address_months_count      1000000 non-null  int64  
 5   customer_age                      1000000 non-null  int64  
 6   days_since_request                1000000 non-null  float64
 7   intended_balcon_amount            1000000 non-null  float64
 8   payment_type                      1000000 non-null  object 
 9   zip_count_4w                      1000000 non-null  int64  
 10  velocity_6h                       1000000 non-null  float64
 11  velocity_24h                      1000

array(['AC', 'AA', 'AB', 'AD', 'AE'], dtype=object)

In [23]:
model_base1 = "/content/model_Base_4.pkl"
model_base2 = "/content/model_Base_7.pkl"
model_variant1_1 = "/content/model_Variant I_4.pkl"
model_variant1_2 = "/content/model_Variant I_7.pkl"

modelb1 = joblib.load(model_base1)
modelb2 = joblib.load(model_base2)
modelv11 = joblib.load(model_variant1_1)
modelv12 = joblib.load(model_variant1_2)
